In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/My Drive/AWA/AWA_Final

!nvcc --version
'''
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1
'''
%tensorflow_version 1.x



#!pip install tensorflow-gpu==1.12.0


import tensorflow as tf
print(tf.__version__)


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


/content/drive/My Drive/AWA/AWA_Final
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
TensorFlow 1.x selected.
1.15.2
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17415653005493388501
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17470780873586467861
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 914299420489647391
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14912199066
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5544916511478971503
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


#Config

In [4]:
AWA_type = 'UAWA'
tau_high = 0.30
tau_low = 0.05
OH = 0.50
exp_num = 1

#Load Dataset

In [5]:
import pickle
import numpy as np
import os
import random
TRACE_LENGTH = 2000
num_sampels_in_Gs_training = 400
DF_NB_CLASSES = 95
def dir_to_burst(dir_data):
    burst_data = np.zeros(dir_data.shape)
    for i in range(len(dir_data)):
        k = 0
        dir_sign = dir_data[i][0]
        for j in range(TRACE_LENGTH):
            if dir_data[i][j] == 0:
                break
            if dir_data[i][j] != dir_sign:
                k += 1
                dir_sign = dir_data[i][j]
            burst_data[i][k] += dir_data[i][j]
    return burst_data


def LoadDataNoDefCW():

    print("Loading non-defended dataset for closed-world scenario")
    # Point to the directory storing data


    # X represents a sequence of traffic directions
    # y represents a sequence of corresponding label (website's label)

    # Load training data
    with open(dataset_dir + 'X_train_NoDef.pkl', 'rb') as handle:
        X_train = np.array(pickle.load(handle, encoding="latin1"))
    with open(dataset_dir + 'y_train_NoDef.pkl', 'rb') as handle:
        y_train = np.array(pickle.load(handle, encoding="latin1"))

    # Load validation data
    with open(dataset_dir + 'X_valid_NoDef.pkl', 'rb') as handle:
        X_valid = np.array(pickle.load(handle, encoding="latin1"))
    with open(dataset_dir + 'y_valid_NoDef.pkl', 'rb') as handle:
        y_valid = np.array(pickle.load(handle, encoding="latin1"))

    # Load testing data
    with open(dataset_dir + 'X_test_NoDef.pkl', 'rb') as handle:
        X_test = np.array(pickle.load(handle, encoding="latin1"))
    with open(dataset_dir + 'y_test_NoDef.pkl', 'rb') as handle:
        y_test = np.array(pickle.load(handle, encoding="latin1"))
    
    y_valid = np.eye(DF_NB_CLASSES)[y_valid]
    y_train = np.eye(DF_NB_CLASSES)[y_train]
    y_test = np.eye(DF_NB_CLASSES)[y_test]

    X_train = X_train[:, :, np.newaxis]
    X_valid = X_valid[:, :, np.newaxis]
    X_test = X_test[:, :, np.newaxis]

    print("Data dimensions:")
    print("X: Training data's shape : ", X_train.shape)
    print("y: Training data's shape : ", y_train.shape)
    print("X: Validation data's shape : ", X_valid.shape)
    print("y: Validation data's shape : ", y_valid.shape)
    print("X: Testing data's shape : ", X_test.shape)
    print("y: Testing data's shape : ", y_test.shape)

    return dir_to_burst(X_train), y_train, dir_to_burst(X_valid), y_valid, dir_to_burst(X_test), y_test

class DFDataLoader():
    """An image loader that uses just a few ImageNet-like images.
    In the actual paper, we used real ImageNet images, but we can't include them
    here because of licensing issues.
    """

    def __init__(self,nb_classes_df,load_burst_data=1):
        if load_burst_data == 0:
            self.train_data,self.train_labels,self.validation_data,self.validation_labels,self.test_data,self.test_labels = LoadDataNoDefCW()
            np.savez('burst_data.npz', train_x=self.train_data, train_y=self.train_labels, val_x = self.validation_data, val_y = self.validation_labels, test_x = self.test_data, test_y = self.test_labels)
        else:
            burst_data = np.load('burst_data.npz')
            train_data = data=burst_data['train_x'][:,:TRACE_LENGTH] 
            train_labels = burst_data['train_y']
            self.validation_data = data=burst_data['val_x'][:,:TRACE_LENGTH] 
            self.validation_labels = burst_data['val_y']
            self.test_data = data=burst_data['test_x'][:,:TRACE_LENGTH] 
            self.test_labels = burst_data['test_y']
            transformer_train_x = []
            transformer_train_y = []
            user_train_x = []
            user_train_y = []
            for i in range(nb_classes_df):
                data_x = train_data[np.argmax(train_labels,axis=1) == i]
                data_y = train_labels[np.argmax(train_labels,axis=1) == i]
                transformer_train_x.append(data_x[:400])
                transformer_train_y.append(data_y[:400])
                user_train_x.append(data_x[400:])
                user_train_y.append(data_y[400:])
            self.transformer_train_x = np.array(transformer_train_x).reshape(nb_classes_df*400,TRACE_LENGTH,1)
            self.transformer_train_y = np.array(transformer_train_y).reshape(nb_classes_df*400,nb_classes_df)
            self.user_train_x = np.array(user_train_x).reshape(nb_classes_df*400,TRACE_LENGTH,1)
            self.user_train_y = np.array(user_train_y).reshape(nb_classes_df*400,nb_classes_df)
        self.nb_classes_df = nb_classes_df


    def get_DF_data(self,data_type,batch_size=None,target_cls=None,src_cls=None):
        assert bool(target_cls != None) != bool(src_cls != None), "BAD func call"
        if data_type == 'transformer_train':
            data = self.transformer_train_x
            label = self.transformer_train_y
        elif data_type == 'user_train':
            data = self.user_train_x
            label = self.user_train_y
        elif data_type == 'validation':
            data = self.validation_data
            label = self.validation_labels
        elif data_type == 'test':
            data = self.test_data
            label = self.test_labels
        else:
            print('Please set the data_type variable')
        if target_cls != None:
            other_class_data = data[np.argmax(label,axis=1) != target_cls]
            other_class_labels = label[np.argmax(label,axis=1) != target_cls]
            if batch_size != None:
                sample = random.sample(list(np.arange(other_class_data.shape[0])), batch_size)
                return other_class_data[sample],other_class_labels[sample]
            return other_class_data,other_class_labels
        if src_cls != None:
            src_class_data = data[np.argmax(label,axis=1) == src_cls]
            src_class_labels = label[np.argmax(label,axis=1) == src_cls]
            if batch_size != None:
                sample = random.sample(list(np.arange(src_class_data.shape[0])), batch_size)
                return src_class_data[sample],src_class_labels[sample]
            return src_class_data,src_class_labels

class AT_Dataset():
    def __init__(self):
        self.train_data = 0
        self.train_labels = 0
        self.validation_data = 0
        self.validation_labels = 0
        self.test_data = 0
        self.test_labels = 0

Trace_loader = DFDataLoader(DF_NB_CLASSES)

# Utils

In [6]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from  tensorflow.python.keras.layers import Input, Conv1D, Activation, BatchNormalization, Lambda, Conv2DTranspose, LeakyReLU, Dropout, Flatten, Dense, ELU, MaxPooling1D, Reshape, Average, ReLU, Lambda
from tensorflow.python.keras.initializers import glorot_uniform
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.data import Dataset


NB_CLASSES = 95 # number of outputs = number of classes



def loss_plot(g1_l,g2_l,d1_l,d2_l=None,path=None, ind = None):
    x = np.arange(len(g1_l))
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.title("Loss graph")
    plt.plot(x,np.array(g1_l),label = "G1_Loss")
    plt.plot(x,g2_l,label = "G2_Loss")
    if d1_l != None:
        plt.plot(x,d1_l,label = "D_Loss")
    if d2_l != None:
        plt.plot(x,d2_l,label = "D2_Loss")
    plt.legend()
    if path == None:
        plt.show()
    else:
        plt.savefig(path+'loss_'+str(ind))
    plt.close()

import PIL.Image
def visualize(trace_len,src=None,trg=None,g1=None,g2=None,path=None,ind=None):
    src = np.round(np.sum(src,axis=0) / len(src)).reshape(1,trace_len,1)
    trg = np.round(np.sum(trg,axis=0) / len(trg)).reshape(1,trace_len,1)
    g1 = np.round(np.sum(g1,axis=0) / len(g1)).reshape(1,trace_len,1)
    g2 = np.round(np.sum(g2,axis=0) / len(g2)).reshape(1,trace_len,1)
    plt.plot(src[0],'.', color='blue')
    plt.plot(trg[0],'.', color='red')
    if True:
        plt.plot(g1[0],'.', color='red')
        plt.plot(g2[0],'.', color='blue')
    plt.plot(np.zeros(src.shape)[0],'.', color='black')
    plt.ylabel('value')
    plt.xlabel('ind')
    if True:
        plt.legend(['src', 'trg','g1','g2'], loc='lower right')

 #   plt.legend([str(c1),str(c2),'zero'], loc='lower right')
    if path == None:
        plt.show()
    else:
        plt.savefig(path+ 'vis_'+str(ind))
    plt.close()
    
def my_sigmoid(x):
    return 1 / (1 + np.exp(-x))

def print_overhead(name="",orginal_data=None,manipulated_data=None, pr = 0):
    oh = np.round((np.sum(np.abs(manipulated_data) - np.abs(orginal_data)) / np.sum(np.abs(orginal_data))) * 100,2)
    if pr == 1:
        print(name,"overhead",oh )
    return oh

def oh_acc_plot(oh1,oh2,oh3,oh4,acc_list,path=None, ind = None):
    x_oh = np.array(oh1)[:,1]
    if len(acc_list) > 0:
        x_acc = np.array(acc_list)[:,1]
    plt.xlabel("Step")
    plt.title("Overhead vs Accuracy under 70%")
    plt.plot(x_oh,np.array(oh1)[:,0],label = "G1_OH_train")
    plt.plot(x_oh,np.array(oh2)[:,0],label = "G1_OH_test")
    plt.plot(x_oh,np.array(oh3)[:,0],label = "G2_OH_train")
    plt.plot(x_oh,np.array(oh4)[:,0],label = "G2_OH_test")
    plt.plot(x_oh,np.ones(len(x_oh))*20,label = "Baseline",color='red')
    if len(acc_list) > 0:
        plt.plot(x_acc,np.array(acc_list)[:,0],label = "Acc < 70%", marker='o')
    plt.legend()
    if path == None:
        plt.show()
    else:
        plt.savefig(path+ 'OH_Acc_'+str(ind))
    plt.close()

class ct1d():
    @staticmethod
    def Conv1DTranspose(model, filters=16, kernel_size=3, strides=2, padding='same'):
        model.add(Lambda(lambda x: tf.expand_dims(x, axis=2)))
        model.add(Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding))
        model.add(Lambda(lambda x: tf.squeeze(x, axis=2)))

ct = ct1d()

class data_class():
        
    def set_information(self,src_cls,trg_cls,oh_src_train,oh_trg_train):
        self.src_cls = src_cls
        self.trg_cls = trg_cls

        self.oh_src_train = oh_src_train
        self.oh_trg_train = oh_trg_train


    def set_data(self,clean_src_trans_train,g_src_trans_train,clean_trg_trans_train,g_trg_trans_train,clean_src_user_train,g_src_user_train,clean_trg_user_train,g_trg_user_train,
                 clean_src_valid,g_src_valid,clean_trg_valid,g_trg_valid,clean_src_test,g_src_test,clean_trg_test,g_trg_test):
        self.clean_src_trans_train = clean_src_trans_train
        self.g_src_trans_train = g_src_trans_train
        self.clean_trg_trans_train = clean_trg_trans_train
        self.g_trg_trans_train = g_trg_trans_train

        self.clean_src_user_train = clean_src_user_train
        self.g_src_user_train = g_src_user_train
        self.clean_trg_user_train = clean_trg_user_train
        self.g_trg_user_train = g_trg_user_train

        self.clean_src_valid = clean_src_valid
        self.g_src_valid = g_src_valid
        self.clean_trg_valid = clean_trg_valid
        self.g_trg_valid = g_trg_valid

        self.clean_src_test = clean_src_test
        self.g_src_test = g_src_test
        self.clean_trg_test = clean_trg_test
        self.g_trg_test = g_trg_test

#Auxiliary Classifier Training

In [7]:
from tensorflow.python.keras.optimizers import Adamax
batch_size = 128
def AC_train(data, file_name, input_shape, classes, num_epochs=50, batch_size=128, train_temp=1, init=None,test_flag = 0,verbose=0,plot_flag=0,load_flag = 1,model_name=None):

        
    model = tf.keras.Sequential()

    filter_num = ['None',32,32,64,64]
    kernel_size = ['None',8,8,8,8]
    conv_stride_size = ['None',1,1,1,1]
    pool_stride_size = ['None',4,4,4,4]
    pool_size = ['None',8,8,8,8]

    model.add(Conv1D(filters=filter_num[1], kernel_size=kernel_size[1], input_shape=input_shape,
                         strides=conv_stride_size[1], padding='same',
                         name='ACblock1_conv1'))
    model.add(ELU(alpha=1.0, name='ACblock1_adv_act1'))

    model.add(Conv1D(filters=filter_num[2], kernel_size=kernel_size[2],
                         strides=conv_stride_size[2], padding='same',
                         name='ACblock2_conv1'))
    model.add(ELU(alpha=1.0, name='ACblock1_adv_act2'))
    model.add(MaxPooling1D(pool_size=pool_size[1], strides=pool_stride_size[1],
                               padding='same', name='ACblock1_pool'))

    model.add(Conv1D(filters=filter_num[3], kernel_size=kernel_size[3],
                         strides=conv_stride_size[3], padding='same',
                         name='ACblock3_conv1'))
    model.add(ELU(alpha=1.0, name='ACblock2_adv_act1'))


    model.add(Conv1D(filters=filter_num[4], kernel_size=kernel_size[4],
                         strides=conv_stride_size[4], padding='same',
                         name='ACblock4_conv1'))
    model.add(ELU(alpha=1.0, name='ACblock2_adv_act2'))
    model.add(MaxPooling1D(pool_size=pool_size[2], strides=pool_stride_size[3],
                               padding='same', name='ACblock2_pool'))

    model.add(Flatten(name='ACflatten'))
    model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name='ACfc1'))
    model.add(Activation('relu', name='ACfc1_act'))
    model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name='ACfc2'))
    model.add(Activation('relu', name='ACfc2_act'))
    model.add(Dense(classes, kernel_initializer=glorot_uniform(seed=0), name='ACfc3'))
    model.add(Activation('softmax', name="softmax"))

    if init != None:
        model.load_weights(init)

    def fn(correct, predicted):
        return tf.nn.softmax_cross_entropy_with_logits(labels=correct,
                                                       logits=predicted / train_temp)

    OPTIMIZER = tf.train.AdamOptimizer(learning_rate=0.0002, beta1=0.9, beta2=0.999, epsilon=1e-08) # Optimizer

    best_model_save = ModelCheckpoint('./'+model_name+'.hdf5', monitor='val_loss', verbose=0, save_best_only=True,
                                          save_weights_only=True, mode='auto', period=1)
    model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,metrics=["accuracy"])
    

    if load_flag == 1:
        print("Loading AC Model!!!")
        model.load_weights(model_name+'.hdf5')
        if test_flag == 1:
            score_test = model.evaluate(data.test_data, data.test_labels, verbose=verbose)
            y_pred = np.argmax(model.predict(data.test_data), axis=1)
            print('Confusion Matrix')
            print(confusion_matrix(np.argmax(data.test_labels, axis=1), y_pred))
            print("Testing accuracy:", score_test[1])
            print(classification_report(np.argmax(data.test_labels, axis=1), y_pred))
        return model

    print("train:",data.transformer_train_x.shape,"valid:",data.transformer_train_x.shape,"test:",data.test_data.shape)
    if test_flag == 1:
        history = model.fit(data.transformer_train_x, data.transformer_train_y,batch_size=batch_size, epochs=num_epochs,verbose=verbose, validation_data=(data.transformer_train_x, data.transformer_train_y), callbacks=[best_model_save])
    else:
        history = model.fit(data.transformer_train_x, data.transformer_train_y,batch_size=batch_size, epochs=num_epochs,verbose=verbose, validation_data=(data.transformer_train_x, data.transformer_train_y))
    
    if test_flag == 1:
        print("Load Best Model")
        model.load_weights(model_name+'.hdf5')

        # Start evaluating model with testing data
        score_test = model.evaluate(data.test_data, data.test_labels, verbose=verbose)
        y_pred = np.argmax(model.predict(data.test_data), axis=1)
        print('Confusion Matrix')
        print(confusion_matrix(np.argmax(data.test_labels, axis=1), y_pred))
        print("Testing accuracy:", score_test[1])
        print(classification_report(np.argmax(data.test_labels, axis=1), y_pred))

    if file_name != None:
        model.save(file_name)
    
    return model

AC = AC_train(Trace_loader,file_name=None,input_shape=(TRACE_LENGTH,1), classes=DF_NB_CLASSES, num_epochs=30,test_flag = 1,verbose=2,plot_flag=1,load_flag = 1,model_name='AC')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Loading AC Model!!!
9500/9500 - 8s - loss: 0.3897 - acc: 0.9407
Confusion Matrix
[[97  0  0 ...  0  0  0]
 [ 1 94  1 ...  0  0  0]
 [ 0  0 94 ...  0  0  0]
 ...
 [ 0  0  0 ... 95  0  0]
 [ 0  0  0 ...  0 92  0]
 [ 0  0  1 ...  0  0 90]]
Testing accuracy: 0.94073683
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       100
           1       0.98      0.94      0.96       100
           2       0.89      0.94      0.91       100
           3       0.97      0.98      0.98       100
           4       0.97      0.98      0.98       100
           5       0.95      0.98      0.97       100
           6       0.84      0.87      0.85       100
           7       0.98      0.99      0.99       100
           8       0.99      0.98      0.98       100
           9       0.94      0.92      0.93       100
          10       0.94      0.94      0.94       100
    

#AWA Framework

In [8]:
def AC_layers(layer_names):
    """ Creates a AC model that returns a list of intermediate output values."""
    # Load our model. Load pretrained AC, trained on WF data
    AC = AC_train(Trace_loader,file_name=None,input_shape=(TRACE_LENGTH,1), classes=DF_NB_CLASSES, num_epochs=30,test_flag = 0,verbose=2,plot_flag=1,load_flag = 1,model_name='AC')
    AC.trainable = False
  
    outputs = [AC.get_layer(name).output for name in layer_names]

    model = tf.keras.Model([AC.input], outputs)
    return model
	
	


class LogitModel(tf.keras.models.Model):
  def __init__(self, logit_layer):
    super(LogitModel, self).__init__()
    self.AC =  AC_layers(logit_layer)
    self.logit_layer = logit_layer
    self.AC.trainable = False

  def call(self, inputs):
    outputs = self.AC(inputs)
    return outputs

def cal_logit_loss(logit_outputs,src=None):
    logit_loss = logit_weight * tf.reduce_mean(tf.maximum(logit_outputs[:,src],0))
    return logit_loss
	
class AWA_Class:
    def __init__(self, trace_len=None,logit_layer=None):
        self.session = tf.InteractiveSession()
        self.max_burst_trace_len = trace_len
        sign_vector = np.ones([trace_len,1]) * -1
        sign_vector[::2] = 1
        self.sign_vector = tf.convert_to_tensor(sign_vector, dtype=tf.float64)
        self.max_trace_len = tf.convert_to_tensor(trace_len, dtype=tf.float64)


        with tf.name_scope('placeholders'):
            self.cls_i_data = tf.placeholder(tf.float32, [None, self.max_burst_trace_len, 1], name="x_class_i")
            self.cls_j_data = tf.placeholder(tf.float32, [None, self.max_burst_trace_len, 1], name="x_class_j")
            self.noise_i = tf.placeholder(tf.float32, [None, self.max_burst_trace_len, 1], name="noise_i")
            self.noise_j = tf.placeholder(tf.float32, [None, self.max_burst_trace_len, 1], name="noise_j")
            self.src_1 = tf.placeholder(tf.int32, shape=(), name= "src_1")
            self.src_2 = tf.placeholder(tf.int32, shape=(), name= "src_2")

        self.logit_extractor = LogitModel(logit_layer)

        self.generator1 = self.make_generator1()
        self.generator2 = self.make_generator2()
        self.discriminator = self.make_discriminator()

        if AWA_type == 'UAWA':
            self.generated_1 = self.generator1(self.noise_i)
            self.generated_2 = self.generator2(self.noise_j)
        elif AWA_type == 'NUAWA':
            self.generated_1 = self.generator1(self.cls_i_data)
            self.generated_2 = self.generator2(self.cls_j_data)          
        self.adjusted_generated_1 = self.adjust_WF_data(self.cls_i_data,self.generated_1)
        self.adjusted_generated_2 = self.adjust_WF_data(self.cls_j_data,self.generated_2)
        self.d_class_i = self.discriminator(self.adjusted_generated_1)
        self.d_class_j = self.discriminator(self.adjusted_generated_2)
        self.d_loss = self.discriminator_loss(self.d_class_i, self.d_class_j)

        self.padded_adjusted_generated_1 = tf.pad(self.adjusted_generated_1,tf.constant([[0,0], [0, TRACE_LENGTH-self.max_burst_trace_len], [0,0]]), mode='CONSTANT', constant_values=0 )
        self.logit_outputs_1 = self.logit_extractor(self.padded_adjusted_generated_1)
        self.logit_loss_1 = cal_logit_loss(self.logit_outputs_1,src=self.src_1)
        self.g1_loss_org , self.g1_oh_loss = self.generator1_loss(self.adjusted_generated_1,self.cls_i_data,self.d_class_i)
        self.g1_loss = self.g1_loss_org + self.g1_oh_loss + self.logit_loss_1


        self.padded_adjusted_generated_2 = tf.pad(self.adjusted_generated_2,tf.constant([[0,0], [0, TRACE_LENGTH-self.max_burst_trace_len], [0,0]]), mode='CONSTANT', constant_values=0 )
        self.logit_outputs_2 = self.logit_extractor(self.padded_adjusted_generated_2)
        self.logit_loss_2 = cal_logit_loss(self.logit_outputs_2,src=self.src_2) 
        self.g2_loss_org , self.g2_oh_loss = self.generator2_loss(self.adjusted_generated_2,self.cls_j_data,self.d_class_j)
        self.g2_loss = self.g2_loss_org + self.g2_oh_loss  + self.logit_loss_2

        with tf.name_scope('optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
            g1_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='generator1')
            self.g1_train = optimizer.minimize(self.g1_loss, var_list=g1_vars)
            g2_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='generator2')
            self.g2_train = optimizer.minimize(self.g2_loss, var_list=g2_vars)
            d_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='discriminator')
            self.d_train = optimizer.minimize(self.d_loss, var_list=d_vars)
        
        tf.global_variables_initializer().run()


  
    def adjust_WF_data(self,x = None,perturbation = None,sign_vector = None):
        mask = tf.expand_dims(tf.tile(tf.cast(tf.minimum(tf.sign(x[:,0]),0) * -1,tf.float32),multiples=[1,self.max_burst_trace_len]),2)   
        pert_rolled = tf.roll(tf.pad(perturbation,[[0,0],[0,1],[0,0]]),shift=1,axis=1)[:,:self.max_burst_trace_len] 
        return x + ( (mask * pert_rolled + (1 - mask) * perturbation) * tf.sign(x) ) 

    def discriminator_loss(self, d_class_i, d_class_j):
        d_loss_i = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_class_i,labels=tf.ones_like(d_class_i)))
        d_loss_j = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_class_j,labels=tf.zeros_like(d_class_j)))
        d_loss = d_loss_i + d_loss_j
        return d_loss

    def generator1_loss(self,perturbation_i=None,x_class_i=None,d_class_i=None):
        g1_oh_loss = tf.maximum(tf.reduce_mean(tf.math.divide(tf.reduce_sum(tf.abs(x_class_i - perturbation_i)),tf.reduce_sum(tf.abs(x_class_i)))) - tau_high, 0) 
        g1_oh_loss_new = tf.minimum(tf.reduce_mean(tf.math.divide(tf.reduce_sum(tf.abs(x_class_i - perturbation_i)),tf.reduce_sum(tf.abs(x_class_i)))) - tau_low, 0) * -1
        g1_loss_org = -1 * (tf.reduce_mean(1 / 2 * tf.math.log(tf.math.sigmoid(d_class_i) + 0.0000001)) + tf.reduce_mean(1 / 2 * tf.math.log(1 - tf.math.sigmoid(d_class_i) + 0.0000001)))
        return disc_weight * g1_loss_org , oh_weight * (g1_oh_loss + g1_oh_loss_new)

    def generator2_loss(self,perturbation_j=None,x_class_j=None,d_class_j=None):
        g2_oh_loss = tf.maximum(tf.reduce_mean(tf.math.divide(tf.reduce_sum(tf.abs(x_class_j - perturbation_j)),tf.reduce_sum(tf.abs(x_class_j)))) - tau_high, 0) 
        g2_oh_loss_new = tf.minimum(tf.reduce_mean(tf.math.divide(tf.reduce_sum(tf.abs(x_class_j - perturbation_j)),tf.reduce_sum(tf.abs(x_class_j)))) - tau_low, 0) * -1
        g2_loss_org = -1 * (tf.reduce_mean(1 / 2 * tf.math.log(tf.math.sigmoid(d_class_j) + 0.0000001)) + tf.reduce_mean(1 / 2 * tf.math.log(1 - tf.math.sigmoid(d_class_j) + 0.0000001)))
        return disc_weight * g2_loss_org , oh_weight * (g2_oh_loss + g2_oh_loss_new)

    def make_generator1(self):
        with tf.variable_scope('generator1'):
            model = tf.keras.Sequential()
            # ---------------------------------------------------------------------
            # c3s1-8
            model.add(Conv1D(filters=8, kernel_size=3, strides=1, padding='same',input_shape=(self.max_burst_trace_len,1)))
            model.add(BatchNormalization(momentum=0.8))
            model.add(ELU(alpha=2.0))
            # -----------------------------------------------------------------
            # d16
            model.add(Conv1D(filters=16, kernel_size=3, strides=2, padding='same'))
            model.add(BatchNormalization(momentum=0.8))
            model.add(ELU(alpha=2.0))
    
            # -------------------------------------------------------------------
            # d32
            model.add(Conv1D(filters=32, kernel_size=3, strides=2, padding='same'))
            model.add(BatchNormalization(momentum=0.8))
            model.add(ELU(alpha=2.0))
    
            # ---------------------------------------------------------------------
            # four r32 blocks
            for _ in range(8):
                model.add(Conv1D(filters=32, kernel_size=3, strides=1, padding='same'))
                model.add(BatchNormalization(momentum=0.8))
                model.add(ELU(alpha=2.0))
    
            # -----------------------------------------------------------------------
            # u16
            ct.Conv1DTranspose(model, filters=16, kernel_size=3, strides=2, padding='same')
            model.add(BatchNormalization(momentum=0.8))
            model.add(ELU(alpha=2.0))
            # this below line is for the cases which our input shape is odd !!
            # G = Conv1D(filters=16, kernel_size=2, strides=1, padding='valid')(G)
    
            # ----------------------------------------------------------------------------
            # u8
            ct.Conv1DTranspose(model, filters=8, kernel_size=3, strides=2, padding='same')
            model.add(BatchNormalization(momentum=0.8))
            model.add(ELU(alpha=2.0))
    
            # -----------------------------------------------------------------------------
            # c3s1-3
            model.add(Conv1D(filters=1, kernel_size=3, strides=1, padding='same'))
            model.add(Activation('relu'))
            return model
    
    
    def make_generator2(self):
        with tf.variable_scope('generator2'):
            model = tf.keras.Sequential()
            # ---------------------------------------------------------------------
            # c3s1-8
            model.add(Conv1D(filters=8, kernel_size=3, strides=1, padding='same',input_shape=(self.max_burst_trace_len,1)))
            model.add(BatchNormalization(momentum=0.8))
            model.add(ELU(alpha=2.0))
            # -----------------------------------------------------------------
            # d16
            model.add(Conv1D(filters=16, kernel_size=3, strides=2, padding='same'))
            model.add(BatchNormalization(momentum=0.8))
            model.add(ELU(alpha=2.0))
    
            # -------------------------------------------------------------------
            # d32
            model.add(Conv1D(filters=32, kernel_size=3, strides=2, padding='same'))
            model.add(BatchNormalization(momentum=0.8))
            model.add(ELU(alpha=2.0))
    
            # ---------------------------------------------------------------------
            # four r32 blocks
            for _ in range(8):
                model.add(Conv1D(filters=32, kernel_size=3, strides=1, padding='same'))
                model.add(BatchNormalization(momentum=0.8))
                model.add(ELU(alpha=2.0))
    
            # -----------------------------------------------------------------------
            # u16
            ct.Conv1DTranspose(model, filters=16, kernel_size=3, strides=2, padding='same')
            model.add(BatchNormalization(momentum=0.8))
            model.add(ELU(alpha=2.0))
            # this below line is for the cases which our input shape is odd !!
            # G = Conv1D(filters=16, kernel_size=2, strides=1, padding='valid')(G)
    
            # ----------------------------------------------------------------------------
            # u8
            ct.Conv1DTranspose(model, filters=8, kernel_size=3, strides=2, padding='same')
            model.add(BatchNormalization(momentum=0.8))
            model.add(ELU(alpha=2.0))
    
            # -----------------------------------------------------------------------------
            # c3s1-3
            model.add(Conv1D(filters=1, kernel_size=3, strides=1, padding='same'))
            model.add(Activation('relu'))
            return model
    
    
    def make_discriminator(self):
        with tf.variable_scope('discriminator'):
            model = tf.keras.Sequential()
            filter_num = ['None',32,32,64,64]
            kernel_size = ['None',8,8,8,8]
            conv_stride_size = ['None',1,1,1,1]
            pool_stride_size = ['None',4,4,4,4]
            pool_size = ['None',8,8,8,8]

            model.add(Conv1D(filters=filter_num[1], kernel_size=kernel_size[1], input_shape=(self.max_burst_trace_len,1),
                         strides=conv_stride_size[1], padding='same',
                         name='dblock1_conv1'))
            model.add(ELU(alpha=1.0, name='dblock1_adv_act1'))

            model.add(Conv1D(filters=filter_num[2], kernel_size=kernel_size[2],
                         strides=conv_stride_size[2], padding='same',
                         name='dblock2_conv1'))
            model.add(ELU(alpha=1.0, name='dblock1_adv_act2'))
            model.add(MaxPooling1D(pool_size=pool_size[1], strides=pool_stride_size[1],
                               padding='same', name='dblock1_pool'))

            model.add(Conv1D(filters=filter_num[3], kernel_size=kernel_size[3],
                         strides=conv_stride_size[3], padding='same',
                         name='dblock3_conv1'))
            model.add(ELU(alpha=1.0, name='dblock2_adv_act1'))


            model.add(Conv1D(filters=filter_num[4], kernel_size=kernel_size[4],
                         strides=conv_stride_size[4], padding='same',
                         name='dblock4_conv1'))
            model.add(ELU(alpha=1.0, name='dblock2_adv_act2'))
            model.add(MaxPooling1D(pool_size=pool_size[2], strides=pool_stride_size[3],
                               padding='same', name='dblock2_pool'))

            model.add(Flatten(name='dflatten'))
            model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name='dfc1'))
            model.add(Activation('relu', name='dfc1_act'))
            model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name='dfc2'))
            model.add(Activation('relu', name='dfc2_act'))
            model.add(Dense(1, kernel_initializer=glorot_uniform(seed=0), name='dfc3'))
            return model

#AWA Training

In [ ]:
from tensorflow.python.keras.backend import clear_session
import IPython.display as display


logit_layer = ['ACfc3']

exp_path = './exp'+str(exp_num)+'/'
if not os.path.exists(exp_path):
    os.makedirs(exp_path)

disc_weight=1e2
oh_weight= 1e3
logit_weight = 1e3

iterations = 1000
batch_size = 100
d_iteration = 2
g_iteration = 2


cls_list = np.arange(94)
keys = np.random.permutation(cls_list)
key1 = keys[:47]
key2 = keys[47:94]

np.savez(exp_path+'Keys'+str(datetime.datetime.now()),key1=key1,key2=key2)

print("Pairs",[[i,j] for i,j in zip(key1,key2)])

assert len(key1) == len(key2), "BAD_CONFIG_IN_CLS_LIST"

for cls_index in range(len(key1)):


    g1_loss_plot = []; g2_loss_plot = []; best_result = 0
    d_loss_plot = []; acc_list = []; oh_src_test = []; oh_src_train = []; oh_trg_test = []; oh_trg_train = []
    exper_data = data_class()
    src_cls = key1[cls_index]
    trg_cls = key2[cls_index]
    print("start",str(datetime.datetime.now()),src_cls,'<->',trg_cls)
    directory_path = exp_path+'Best_Gs_Output_'+str(src_cls)+'-->'+str(trg_cls)+'/'#+'('+str(datetime.datetime.now())+')'+'/'
    plot_path = directory_path + 'plot/'
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        os.makedirs(plot_path)
  
    src_trans_train_data,src_trans_train_labels = Trace_loader.get_DF_data(data_type= 'transformer_train', src_cls= src_cls)
    src_user_train_data,src_user_train_labels = Trace_loader.get_DF_data(data_type= 'user_train', src_cls= src_cls)
    src_valid_data,src_valid_labels = Trace_loader.get_DF_data(data_type= 'validation', src_cls= src_cls)
    src_test_data,src_test_labels = Trace_loader.get_DF_data(data_type= 'test', src_cls= src_cls)
    trg_trans_train_data,trg_trans_train_labels = Trace_loader.get_DF_data(data_type= 'transformer_train', src_cls= trg_cls) 
    trg_user_train_data,trg_user_train_labels = Trace_loader.get_DF_data(data_type= 'user_train', src_cls= trg_cls) 
    trg_valid_data,trg_valid_labels = Trace_loader.get_DF_data(data_type= 'validation', src_cls= trg_cls) 
    trg_test_data,trg_test_labels = Trace_loader.get_DF_data(data_type= 'test', src_cls= trg_cls)
  

    max_burst_trace_len = 2000#max([max(np.array([ np.count_nonzero(i) for i in src_trans_train_data])),max(np.array([ np.count_nonzero(i) for i in trg_trans_train_data]))]) 
    if max_burst_trace_len % 4 != 0:
        max_burst_trace_len += 4 - (max_burst_trace_len % 4)

    src_trans_train_data = src_trans_train_data[:,:max_burst_trace_len]
    src_user_train_data = src_user_train_data[:,:max_burst_trace_len]
    src_valid_data = src_valid_data[:,:max_burst_trace_len]
    src_test_data = src_test_data[:,:max_burst_trace_len]
    trg_trans_train_data = trg_trans_train_data[:,:max_burst_trace_len]
    trg_user_train_data = trg_user_train_data[:,:max_burst_trace_len]
    trg_valid_data = trg_valid_data[:,:max_burst_trace_len]
    trg_test_data = trg_test_data[:,:max_burst_trace_len]

    #max_burst_vector = np.max(np.append(np.abs(src_train_data),np.abs(trg_train_data),axis=0),axis=0)[:max_burst_trace_len]
    print(src_trans_train_data.shape,src_user_train_data.shape,src_test_data.shape,src_valid_data.shape,trg_trans_train_data.shape,trg_user_train_data.shape,trg_test_data.shape,trg_valid_data.shape)


    clear_session()
    awa = AWA_Class(trace_len=max_burst_trace_len, logit_layer=logit_layer)
    trnsformers_selecting_flag = 0
    for i in range(iterations):
        t_g1_lor = 0
        t_g2_lor = 0
        t_d_l = 0
        for d_i in range(d_iteration):
            sample_index = np.random.randint(len(src_trans_train_data), size=batch_size)
            batch_src_x = src_trans_train_data[sample_index]
            batch_src_noise = np.random.normal(size=batch_src_x.shape)
            sample_index = np.random.randint(len(trg_trans_train_data), size=batch_size)
            batch_trg_x = trg_trans_train_data[sample_index]
            batch_trg_noise = np.random.normal(size=batch_trg_x.shape)

            _, d_l = awa.session.run([awa.d_train,awa.d_loss], feed_dict={awa.cls_i_data: batch_src_x, awa.cls_j_data: batch_trg_x, awa.noise_i: batch_src_noise, awa.noise_j: batch_trg_noise})
            t_d_l += d_l
        d_loss_plot.append(t_d_l/d_iteration)
        
        for g1_i in range(g_iteration):
            sample_index = np.random.randint(len(src_trans_train_data), size=batch_size)
            batch_src_x = src_trans_train_data[sample_index]
            batch_src_noise = np.random.normal(size=batch_src_x.shape)

            _,g1_l, g1_lor, g1_loh, l1_loss, d1_out,pert,adj_new_data = awa.session.run([awa.g1_train,awa.g1_loss,awa.g1_loss_org,awa.g1_oh_loss, awa.logit_loss_1 ,awa.d_class_i, awa.generated_1, awa.adjusted_generated_1], 
																					feed_dict={awa.cls_i_data: batch_src_x, awa.src_1: src_cls, awa.noise_i: batch_src_noise })
            assert np.sum(pert < 0) == 0, "Health issue in perturbation"
            assert np.sum((np.abs(adj_new_data) - np.abs(batch_src_x)) < 0) == 0, "Health issue in sign" 
            assert np.array_equal(np.array([np.sum(np.abs(np.sign(i))) for i in adj_new_data ]) , np.array([np.sum(np.abs(np.sign(i))) for i in batch_src_x ])), "Health issue in size of trace"
            t_g1_lor += (g1_lor /g_iteration)
        g1_loss_plot.append(t_g1_lor)

        for d_i in range(d_iteration):
            sample_index = np.random.randint(len(src_trans_train_data), size=batch_size)
            batch_src_x = src_trans_train_data[sample_index]
            batch_src_noise = np.random.normal(size=batch_src_x.shape)
            sample_index = np.random.randint(len(trg_trans_train_data), size=batch_size)
            batch_trg_x = trg_trans_train_data[sample_index]
            batch_trg_noise = np.random.normal(size=batch_trg_x.shape)

            _, d_l = awa.session.run([awa.d_train,awa.d_loss], feed_dict={awa.cls_i_data: batch_src_x, awa.cls_j_data: batch_trg_x, awa.noise_i: batch_src_noise, awa.noise_j: batch_trg_noise})

        for g2_i in range(g_iteration):
            sample_index = np.random.randint(len(trg_trans_train_data), size=batch_size)
            batch_trg_x = trg_trans_train_data[sample_index]
            batch_trg_noise = np.random.normal(size=batch_trg_x.shape)

            _,g2_l, g2_lor,g2_loh,  l2_loss, d2_out,pert,adj_new_data = awa.session.run([awa.g2_train,awa.g2_loss,awa.g2_loss_org,awa.g2_oh_loss, awa.logit_loss_2 ,awa.d_class_j, awa.generated_2, awa.adjusted_generated_2], 
																					feed_dict={awa.cls_j_data: batch_trg_x, awa.src_2: trg_cls, awa.noise_j: batch_trg_noise })
            assert np.sum(pert < 0) == 0, "Health issue in perturbation"
            assert np.sum((np.abs(adj_new_data) - np.abs(batch_trg_x)) < 0) == 0, "Health issue in sign"
            assert np.array_equal(np.array([np.sum(np.abs(np.sign(i))) for i in adj_new_data ]) , np.array([np.sum(np.abs(np.sign(i))) for i in batch_trg_x ])), "Health issue in size of trace"
            t_g2_lor += (g2_lor/g_iteration)
        g2_loss_plot.append(t_g2_lor)


        if (i+1) % 50 == 0 or (i+1) % iterations == 0:
            print("*"*50,' ',i+1,"/",iterations) 
            generated_src_trans_train = awa.adjusted_generated_1.eval(feed_dict={awa.cls_i_data: src_trans_train_data, awa.noise_i: np.random.normal(size=src_trans_train_data.shape) })
            assert np.sum((np.abs(generated_src_trans_train) - np.abs(src_trans_train_data)) < 0) == 0, "Health issue in sign1"
            generated_trg_trans_train = awa.adjusted_generated_2.eval(feed_dict={awa.cls_j_data: trg_trans_train_data, awa.noise_j: np.random.normal(size=trg_trans_train_data.shape) })
            assert np.sum((np.abs(generated_trg_trans_train) - np.abs(trg_trans_train_data)) < 0) == 0, "Health issue in sign2"
            
            oh_src_train.append([print_overhead("g1_train",src_trans_train_data,generated_src_trans_train,pr = 1),i])
            oh_trg_train.append([print_overhead("g2_train",trg_trans_train_data,generated_trg_trans_train,pr = 1),i])
            
            print("d_l:",d_l)
            print("g1_l:",g1_l,"g1_lor:", t_g1_lor,"g1_loh:",g1_loh,"l1_loss",l1_loss,"d1_out:",np.mean(my_sigmoid(d1_out)) )
            print("g2_l:",g2_l,"g2_lor:", t_g2_lor,"g2_loh:",g2_loh,"l2_loss",l2_loss,"d2_out:",np.mean(my_sigmoid(d2_out)) )
            if (oh_src_train[-1][0] < OH * 100 and oh_trg_train[-1][0] < OH * 100) or ((i+1) % iterations == 0 and trnsformers_selecting_flag == 0):
                print("Saving data!!!")
                trnsformers_selecting_flag = 1
                generated_src_user_train = awa.adjusted_generated_1.eval(feed_dict={awa.cls_i_data: src_user_train_data, awa.noise_i: np.random.normal(size=src_user_train_data.shape) })
                assert np.sum((np.abs(generated_src_user_train) - np.abs(src_user_train_data)) < 0) == 0, "Health issue in sign11"
                generated_trg_user_train = awa.adjusted_generated_2.eval(feed_dict={awa.cls_j_data: trg_user_train_data, awa.noise_j: np.random.normal(size=trg_user_train_data.shape) })
                assert np.sum((np.abs(generated_trg_user_train) - np.abs(trg_user_train_data)) < 0) == 0, "Health issue in sign22"
            
                oh_src_train.append([print_overhead("g1_user_train",src_user_train_data,generated_src_user_train,pr = 1),i])
                oh_trg_train.append([print_overhead("g2_user_train",trg_user_train_data,generated_trg_user_train,pr = 1),i])

                generated_src_valid = awa.adjusted_generated_1.eval(feed_dict={awa.cls_i_data: src_valid_data, awa.noise_i: np.random.normal(size=src_valid_data.shape) })
                assert np.sum((np.abs(generated_src_valid) - np.abs(src_valid_data)) < 0) == 0, "Health issue in sign3"
                generated_trg_valid = awa.adjusted_generated_2.eval(feed_dict={awa.cls_j_data: trg_valid_data, awa.noise_j: np.random.normal(size=trg_valid_data.shape) })
                assert np.sum((np.abs(generated_trg_valid) - np.abs(trg_valid_data)) < 0) == 0, "Health issue in sign4"

                generated_src_test = awa.adjusted_generated_1.eval(feed_dict={awa.cls_i_data: src_test_data, awa.noise_i: np.random.normal(size=src_test_data.shape) })
                assert np.sum((np.abs(generated_src_test) - np.abs(src_test_data)) < 0) == 0, "Health issue in sign5"          
                generated_trg_test = awa.adjusted_generated_2.eval(feed_dict={awa.cls_j_data: trg_test_data, awa.noise_j: np.random.normal(size=trg_test_data.shape) })
                assert np.sum((np.abs(generated_trg_test) - np.abs(trg_test_data)) < 0) == 0, "Health issue in sign6"

                #visualize(max_burst_trace_len,generated_src_valid,generated_trg_valid,generated_src_valid,generated_trg_valid,path=plot_path,ind=i)
                #loss_plot(g1_loss_plot,g2_loss_plot,d_loss_plot)#,path=plot_path, ind = i)

                exper_data.set_information(src_cls=src_cls,trg_cls=trg_cls,oh_src_train=oh_src_train[-1][0],oh_trg_train=oh_trg_train[-1][0])
                exper_data.set_data(clean_src_trans_train=src_trans_train_data,g_src_trans_train=generated_src_trans_train,clean_trg_trans_train=trg_trans_train_data,g_trg_trans_train=generated_trg_trans_train,
                                    clean_src_user_train=src_user_train_data,g_src_user_train=generated_src_user_train,clean_trg_user_train=trg_user_train_data,g_trg_user_train=generated_trg_user_train,
                                    clean_src_valid=src_valid_data,g_src_valid=generated_src_valid,clean_trg_valid=trg_valid_data,g_trg_valid=generated_trg_valid,
                                    clean_src_test=src_test_data,g_src_test=generated_src_test,clean_trg_test=trg_test_data,g_trg_test=generated_trg_test)
                with open(directory_path+'Best_Gs_Output_'+str(src_cls)+'-->'+str(trg_cls)+'_untar.pkl', 'wb') as output:
                    awa.generator1.save_weights(directory_path+'g1')
                    awa.generator2.save_weights(directory_path+'g2')
                    pickle.dump(exper_data,output)

    print("End",str(datetime.datetime.now()))
    awa.session.close()
    print("Session is closed:", awa.session._closed)
